In [1]:
import pandas as pd
import numpy as np

# Altman Z-Score
1. Features 
    - log총자산(단위사용)
    - log(매출액/총자산)   (= 총자산회전율)
    - 이익잉여금/총자산
    - 자기자본시장가격/총부채

2. 범위별 판단
    - k2 < - 2.3 : 부실가능성 심각
    - -2.3 < k2 < 0.75 : 판정보류
    - k2 > 0.75 : 건전판정

    

# I. DF

## I-I. DF 불러오기

In [2]:
# 연결 ALE
df_a = pd.read_excel('./dataset_2nd/z_score_asset.xlsx')
df_l = pd.read_excel('./dataset_2nd/z_score_liability.xlsx')
df_e = pd.read_excel('./dataset_2nd/z_score_equity.xlsx')

df_ale = pd.concat([df_a, df_l, df_e], axis=1) # 회사명, 거래소코드, 회계년도 제거 필요 

# 연결 Sales & R/E
df_sre = pd.read_excel('./dataset_2nd/z_score_sales_retainedearning.xlsx')

# 개별 ALE
df_ale_each = pd.read_excel('./dataset_2nd/z_score_ALE_each.xlsx')

# 개별 Sales & R/E
df_sre_each = pd.read_excel('./dataset_2nd/z_score_salesRE_each.xlsx')

# 주가 & 발행주식수
df_price = pd.read_excel('./dataset_2nd/z_score_price.xlsx')
df_num_share = pd.read_excel('./dataset_2nd/z_score_sharesnumber.xlsx') # 중복 제거 필요




In [3]:
df_share = pd.read_excel('./dataset_2nd/z_score_share.xlsx')

---

In [4]:
df_share

,회사명,거래소코드,회계년도,[제조]* 발행한 주식총수(*)(IFRS연결)(주),[제조] 보통주(IFRS연결)(주),[제조] 우선주(IFRS연결)(천원),[제조]* 발행한 주식총수(*)(IFRS)(천원),[제조] 보통주(IFRS)(주),[제조] 우선주(IFRS)(주),종가(원)
0,(주)CMG제약,58820,2014/12,86626866.0,86626866.0,0.0,86626866.0,86626866.0,0.0,912.0
1,(주)CMG제약,58820,2018/12,108887244.0,108887244.0,0.0,108887244.0,108887244.0,0.0,4235.0
2,(주)CMG제약,58820,2019/12,138892244.0,138892244.0,0.0,138892244.0,138892244.0,0.0,4250.0
3,(주)CMG제약,58820,2020/12,NaN,NaN,NaN,138892244.0,138892244.0,0.0,5780.0
4,(주)CMG제약,58820,2017/12,108887244.0,108887244.0,0.0,108887244.0,108887244.0,0.0,6240.0
...,...,...,...,...,...,...,...,...,...,...
17482,흥구석유(주),24060,2022/12,NaN,NaN,NaN,15000000.0,15000000.0,0.0,5680.0
17483,흥구석유(주),24060,2013/12,NaN,NaN,NaN,15000000.0,15000000.0,0.0,2000.0
17484,흥구석유(주),24060,2018/12,NaN,NaN,NaN,15000000.0,15000000.0,0.0,2705.0
17485,흥구석유(주),24060,2016/12,NaN,NaN,NaN,15000000.0,15000000.0,0.0,3425.0


---

In [5]:
df_sre

,회사명,거래소코드,회계년도,[제조] 이익잉여금(결손금)(*)(IFRS연결)(천원),[제조]매출액(수익)(*)(IFRS연결)(천원)
0,(주)CMG제약,58820,2011/12,-45215050.0,17751704.0
1,(주)CMG제약,58820,2012/12,-52379991.0,16255389.0
2,(주)CMG제약,58820,2013/12,-52201720.0,20402140.0
3,(주)CMG제약,58820,2014/12,-52393918.0,22752584.0
4,(주)CMG제약,58820,2015/12,-56413575.0,27041894.0
...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,NaN,NaN
17483,흥구석유(주),24060,2019/12,NaN,NaN
17484,흥구석유(주),24060,2020/12,NaN,NaN
17485,흥구석유(주),24060,2021/12,NaN,NaN


## I-II. 발행주식수 DF 중복 제거 
    - row수 : 36297 -> 17487

In [6]:
df_copy = df_num_share.copy()

In [7]:
df_copy

,회사명,거래소코드,회계년도,발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)
0,(주)CMG제약,58820,2011/12,43693200.0
1,(주)CMG제약,58820,2011/12,43693200.0
2,(주)CMG제약,58820,2012/12,66025540.0
3,(주)CMG제약,58820,2012/12,66025540.0
4,(주)CMG제약,58820,2013/12,86626866.0
...,...,...,...,...
36292,흥구석유(주),24060,2020/12,15000000.0
36293,흥구석유(주),24060,2021/12,15000000.0
36294,흥구석유(주),24060,2021/12,15000000.0
36295,흥구석유(주),24060,2022/12,15000000.0


In [8]:
df_num_share_5 = pd.pivot_table(df_copy, index=["회사명","거래소코드","회계년도"], values='발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)', aggfunc=['count','sum','mean'])[[( 'mean', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)')]].reset_index()

In [9]:
df_num_share_5.columns = ['회사명','거래소코드','회계년도', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)']

In [10]:
df_num_share = df_num_share_5

---

## I-III. 개별 ALE DF 정리

In [11]:
df_ale.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]자산(*)(IFRS연결)', '회사명', '거래소코드', '회계년도',
       '[제조]부채(*)(IFRS연결)', '회사명', '거래소코드', '회계년도', '[제조]자본(*)(IFRS연결)(천원)'],
      dtype='object')

In [12]:
df_ale.columns = ['회사명', '거래소코드', '회계년도', '[제조]자산(*)(IFRS연결)', '회사명2', '거래소코드2', '회계년도2',\
       '[제조]부채(*)(IFRS연결)', '회사명3', '거래소코드3', '회계년도3', '[제조]자본(*)(IFRS연결)(천원)']

In [13]:
df_ale.drop(['회사명2', '거래소코드2', '회계년도2','회사명3', '거래소코드3', '회계년도3'], axis = 1, inplace = True)

## I-IV. DF 통합

In [14]:
# 중복 컬럼 제거
df_sre.drop(['회사명','거래소코드', '회계년도'], axis = 1, inplace = True)
df_ale_each.drop(['회사명','거래소코드', '회계년도'], axis = 1, inplace = True)
df_sre_each.drop(['회사명','거래소코드', '회계년도'], axis = 1, inplace = True)
df_price.drop(['회사명','거래소코드', '회계년도'], axis = 1, inplace = True)
df_num_share.drop(['회사명','거래소코드', '회계년도'], axis = 1, inplace = True)

In [15]:
# concat
df = pd.concat([df_ale, df_ale_each, df_sre, df_sre_each, df_price, df_num_share], axis = 1)

---

# II. 연결&개별 합쳐서 결측치 문제 해결

## II-I.연결 결측치에 개별 값으로 대체

### 1. 연+개_자산

In [16]:
df_copy = df.copy()
# df = df_copy.copy()

In [17]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]자산(*)(IFRS연결)', '[제조]부채(*)(IFRS연결)',
       '[제조]자본(*)(IFRS연결)(천원)', '[제조]자산(*)(IFRS)(천원)', '[제조]부채(*)(IFRS)(천원)',
       '[제조]자본(*)(IFRS)(천원)', '[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)',
       '[제조]매출액(수익)(*)(IFRS연결)(천원)', '[제조]   이익잉여금(결손금)(*)(IFRS)(천원)',
       '[제조]매출액(수익)(*)(IFRS)(천원)', '종가(원)',
       '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)'],
      dtype='object')

In [18]:
# 컬럼 제목 정리
df.columns = ['회사명', '거래소코드', '회계년도', 
       '[제조]자산(*)(IFRS연결)(천원)', '[제조]부채(*)(IFRS연결)(천원)','[제조]자본(*)(IFRS연결)(천원)',
       '[제조]자산(*)(IFRS)(천원)', '[제조]부채(*)(IFRS)(천원)','[제조]자본(*)(IFRS)(천원)',
       '[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)','[제조]매출액(수익)(*)(IFRS연결)(천원)',
       '[제조]   이익잉여금(결손금)(*)(IFRS)(천원)','[제조]매출액(수익)(*)(IFRS)(천원)',
       '종가(원)', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)']

In [19]:
df.loc[df['[제조]자산(*)(IFRS연결)(천원)'].isna(), ['회사명','거래소코드','회계년도','[제조]자산(*)(IFRS)(천원)','[제조]자산(*)(IFRS연결)(천원)']]

,회사명,거래소코드,회계년도,[제조]자산(*)(IFRS)(천원),[제조]자산(*)(IFRS연결)(천원)
9,(주)CMG제약,58820,2020/12,195355583.0,NaN
10,(주)CMG제약,58820,2021/12,199174593.0,NaN
11,(주)CMG제약,58820,2022/12,213260376.0,NaN
36,(주)가온칩스,399720,2018/12,NaN,NaN
37,(주)가온칩스,399720,2019/12,NaN,NaN
...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,75205581.0,NaN
17483,흥구석유(주),24060,2019/12,83661499.0,NaN
17484,흥구석유(주),24060,2020/12,83721348.0,NaN
17485,흥구석유(주),24060,2021/12,82647109.0,NaN


In [20]:
a = df.loc[df['[제조]자산(*)(IFRS연결)(천원)'].isna(), ['회사명','거래소코드','회계년도','[제조]자산(*)(IFRS)(천원)','[제조]자산(*)(IFRS연결)(천원)']]['[제조]자산(*)(IFRS)(천원)']

In [21]:
df.loc[df['[제조]자산(*)(IFRS연결)(천원)'].isna(),'[제조]자산(*)(IFRS연결)(천원)'] = a


In [22]:
df['[제조]자산(*)(IFRS연결)(천원)'].isna().sum()

1868

- 연결과 개별 둘다 Nan 인 경우 -> 1868

### 2. 연+개_매출액

In [23]:
b = df.loc[df['[제조]매출액(수익)(*)(IFRS연결)(천원)'].isna(), '[제조]매출액(수익)(*)(IFRS)(천원)']

In [24]:
df['[제조]매출액(수익)(*)(IFRS연결)(천원)'].isna().sum()

6048

In [25]:
df.loc[df['[제조]매출액(수익)(*)(IFRS연결)(천원)'].isna(),'[제조]매출액(수익)(*)(IFRS연결)(천원)'] = b

In [26]:
df['[제조]매출액(수익)(*)(IFRS연결)(천원)'].isna().sum()

1864

### 3. 연+개_이익잉여금

In [27]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]자산(*)(IFRS연결)(천원)',
       '[제조]부채(*)(IFRS연결)(천원)', '[제조]자본(*)(IFRS연결)(천원)', '[제조]자산(*)(IFRS)(천원)',
       '[제조]부채(*)(IFRS)(천원)', '[제조]자본(*)(IFRS)(천원)',
       '[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)', '[제조]매출액(수익)(*)(IFRS연결)(천원)',
       '[제조]   이익잉여금(결손금)(*)(IFRS)(천원)', '[제조]매출액(수익)(*)(IFRS)(천원)', '종가(원)',
       '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)'],
      dtype='object')

In [28]:
c = df.loc[df['[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)'].isna(), '[제조]   이익잉여금(결손금)(*)(IFRS)(천원)']

In [29]:
df.loc[df['[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)'].isna(),'[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)'] = c

In [30]:
df['[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)'].isna().sum()

1864

### 4. 연+개_자본

In [31]:
d = df.loc[df['[제조]자본(*)(IFRS연결)(천원)'].isna(), '[제조]자본(*)(IFRS)(천원)']


In [32]:
df.loc[df['[제조]자본(*)(IFRS연결)(천원)'].isna(), '[제조]자본(*)(IFRS연결)(천원)'] = d

In [33]:
df['[제조]자본(*)(IFRS연결)(천원)'].isna().sum()

1868

### 5. 연+개_부채

In [34]:
e = df.loc[df['[제조]부채(*)(IFRS연결)(천원)'].isna(), '[제조]부채(*)(IFRS)(천원)']

In [35]:
df.loc[df['[제조]부채(*)(IFRS연결)(천원)'].isna(),'[제조]부채(*)(IFRS연결)(천원)'] = e

In [36]:
df['[제조]부채(*)(IFRS연결)(천원)'].isna().sum()

1868

In [37]:
df

,회사명,거래소코드,회계년도,[제조]자산(*)(IFRS연결)(천원),[제조]부채(*)(IFRS연결)(천원),[제조]자본(*)(IFRS연결)(천원),[제조]자산(*)(IFRS)(천원),[제조]부채(*)(IFRS)(천원),[제조]자본(*)(IFRS)(천원),[제조] 이익잉여금(결손금)(*)(IFRS연결)(천원),[제조]매출액(수익)(*)(IFRS연결)(천원),[제조] 이익잉여금(결손금)(*)(IFRS)(천원),[제조]매출액(수익)(*)(IFRS)(천원),종가(원),발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)
0,(주)CMG제약,58820,2011/12,27305198.0,9270949.0,18034249.0,27548539.0,9257624.0,18290915.0,-45215050.0,17751704.0,-44971257.0,17751704.0,785.0,43693200.0
1,(주)CMG제약,58820,2012/12,57108035.0,25517350.0,31590685.0,56988604.0,25386064.0,31602540.0,-52379991.0,16255389.0,-52368787.0,16255389.0,1330.0,66025540.0
2,(주)CMG제약,58820,2013/12,55912563.0,6915281.0,48997281.0,55961620.0,6913407.0,49048213.0,-52201720.0,20402140.0,-52151440.0,20322140.0,932.0,86626866.0
3,(주)CMG제약,58820,2014/12,58068326.0,9065682.0,49002644.0,58127098.0,9060059.0,49067039.0,-52393918.0,22752584.0,-52330174.0,22752584.0,912.0,57751244.0
4,(주)CMG제약,58820,2015/12,56532821.0,9725933.0,46806888.0,56591972.0,9719267.0,46872705.0,-56413575.0,27041894.0,-56348409.0,27041894.0,2110.0,88297244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,75205581.0,4763313.0,70442269.0,75205581.0,4763313.0,70442269.0,34090738.0,166208303.0,34090738.0,166208303.0,2705.0,15000000.0
17483,흥구석유(주),24060,2019/12,83661499.0,6660240.0,77001259.0,83661499.0,6660240.0,77001259.0,40649728.0,150668735.0,40649728.0,150668735.0,5360.0,15000000.0
17484,흥구석유(주),24060,2020/12,83721348.0,4121163.0,79600184.0,83721348.0,4121163.0,79600184.0,43248653.0,114843735.0,43248653.0,114843735.0,7220.0,15000000.0
17485,흥구석유(주),24060,2021/12,82647109.0,3751076.0,78896033.0,82647109.0,3751076.0,78896033.0,42544502.0,132213730.0,42544502.0,132213730.0,7160.0,15000000.0


---

## II-II. 연결 & 개별 Both 결측치인 경우 -> 오뜨카지..

### 1. 갯수확인

In [38]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]자산(*)(IFRS연결)(천원)',
       '[제조]부채(*)(IFRS연결)(천원)', '[제조]자본(*)(IFRS연결)(천원)', '[제조]자산(*)(IFRS)(천원)',
       '[제조]부채(*)(IFRS)(천원)', '[제조]자본(*)(IFRS)(천원)',
       '[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)', '[제조]매출액(수익)(*)(IFRS연결)(천원)',
       '[제조]   이익잉여금(결손금)(*)(IFRS)(천원)', '[제조]매출액(수익)(*)(IFRS)(천원)', '종가(원)',
       '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)'],
      dtype='object')

In [39]:
# 연결 & 개별 모두 결측치인 row 수

    # ALE
print(((df['[제조]자산(*)(IFRS연결)(천원)'].isna()) & (df['[제조]자산(*)(IFRS)(천원)'].isna())).sum())
print(((df['[제조]부채(*)(IFRS연결)(천원)'].isna()) & (df['[제조]부채(*)(IFRS)(천원)'].isna())).sum())
print(((df['[제조]자본(*)(IFRS연결)(천원)'].isna()) & (df['[제조]자본(*)(IFRS)(천원)'].isna())).sum())
    # Sales & R/E
print(((df['[제조]매출액(수익)(*)(IFRS연결)(천원)'].isna()) & (df['[제조]매출액(수익)(*)(IFRS)(천원)'].isna())).sum())
print(((df['[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)'].isna()) & (df['[제조]   이익잉여금(결손금)(*)(IFRS)(천원)'].isna())).sum())


1868
1868
1868
1864
1864


# III. Altman_K2-Score
- 한국기업 대상으로 도산예측모형
- K2_score : 상장기업 적용 모형 
    - 자기자본 시장가격 : '시가총액'으로 대체 

In [63]:
altman = df.iloc[:, 0:3]

In [64]:
df_copy2 = df.copy()
# df = df_copy2.copy()

## III-I. 변수 확보 

### 1. 기업규모: log(총자산)

In [98]:
# 자산 단위 변경 : 1,000 -> 1,000,000
altman['총자산(log)'] = np.log(df['[제조]자산(*)(IFRS연결)(천원)']/1000)

c:\Users\zino0\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### 2. 총자산회전율 : log(매출액/총자산) 

In [99]:
altman['총자산회전율(log)'] = np.log(df['[제조]매출액(수익)(*)(IFRS연결)(천원)']/df['[제조]자산(*)(IFRS연결)(천원)'])

### 3. 누적수익성 : 이익잉여금/총자산 

In [100]:
altman['누적수익성'] = df['[제조]      이익잉여금(결손금)(*)(IFRS연결)(천원)']/df['[제조]자산(*)(IFRS연결)(천원)']

### 4. 재무구조 : 자기자본 시장가격/총부채 

### share DF 
1. 연결 주식총수 = Nan 일 때, 개별 주식총수로 대체
2. 우선주 + 보통주 = 주식총수 True 여부 확인
3. 종가 결측치 확인 
4. 주식총수 x 종가    

In [101]:
df_share.isna().sum()

회사명                                0
거래소코드                              0
회계년도                               0
[제조]* 발행한 주식총수(*)(IFRS연결)(주)    1875
[제조]   보통주(IFRS연결)(주)           6048
[제조]   우선주(IFRS연결)(천원)          6048
[제조]* 발행한 주식총수(*)(IFRS)(천원)     1993
[제조]   보통주(IFRS)(주)             1993
[제조]   우선주(IFRS)(주)             1993
종가(원)                           3237
시가총액                            3478
dtype: int64

In [102]:
# 1. 연결 주식총수 nan값 개별로 대체
f = df_share.loc[df_share['[제조]* 발행한 주식총수(*)(IFRS연결)(주)'].isna()]['[제조]* 발행한 주식총수(*)(IFRS)(천원)']

In [103]:
df_share.loc[df_share['[제조]* 발행한 주식총수(*)(IFRS연결)(주)'].isna(),'[제조]* 발행한 주식총수(*)(IFRS연결)(주)'] = f

In [104]:
df_share.isna().sum()

회사명                                0
거래소코드                              0
회계년도                               0
[제조]* 발행한 주식총수(*)(IFRS연결)(주)    1875
[제조]   보통주(IFRS연결)(주)           6048
[제조]   우선주(IFRS연결)(천원)          6048
[제조]* 발행한 주식총수(*)(IFRS)(천원)     1993
[제조]   보통주(IFRS)(주)             1993
[제조]   우선주(IFRS)(주)             1993
종가(원)                           3237
시가총액                            3478
dtype: int64

In [105]:
df_share.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]* 발행한 주식총수(*)(IFRS연결)(주)',
       '[제조]   보통주(IFRS연결)(주)', '[제조]   우선주(IFRS연결)(천원)',
       '[제조]* 발행한 주식총수(*)(IFRS)(천원)', '[제조]   보통주(IFRS)(주)',
       '[제조]   우선주(IFRS)(주)', '종가(원)', '시가총액'],
      dtype='object')

In [106]:
# 2. 우선주 + 보통주 = 주식총수 True 여부 확인
    # 연결 개별 합친 후 주식총수 총합 오류 있는지 여부 확인 
    
share_a = df_share.loc[(df_share['[제조]   보통주(IFRS연결)(주)'] + df_share['[제조]   우선주(IFRS연결)(천원)'] != df_share['[제조]* 발행한 주식총수(*)(IFRS연결)(주)'])\
    &(df_share['[제조]   보통주(IFRS)(주)'] + df_share['[제조]   우선주(IFRS)(주)'] != df_share['[제조]* 발행한 주식총수(*)(IFRS)(천원)'])].index

In [107]:
    # 연결 - 결측치 Index
share_b = df_share.loc[df_share['[제조]* 발행한 주식총수(*)(IFRS연결)(주)'].isna()].index

In [108]:
(share_a != share_b).sum()

0

> 주식총수(연결)의 결측치 자체가 주식총합(!= 보통주 + 우선주)인 이유

- Anyway, 연결_주식총수로 시가총액 계산이 최선 

In [109]:
# 3. 종가 결측치 
df_share['종가(원)'].isna().sum()

3237

In [110]:
df_share.columns

Index(['회사명', '거래소코드', '회계년도', '[제조]* 발행한 주식총수(*)(IFRS연결)(주)',
       '[제조]   보통주(IFRS연결)(주)', '[제조]   우선주(IFRS연결)(천원)',
       '[제조]* 발행한 주식총수(*)(IFRS)(천원)', '[제조]   보통주(IFRS)(주)',
       '[제조]   우선주(IFRS)(주)', '종가(원)', '시가총액'],
      dtype='object')

In [111]:
df_share['시가총액'] = df_share['[제조]* 발행한 주식총수(*)(IFRS연결)(주)']* df_share['종가(원)']

In [112]:
# 시가총액 단위변경 : 1 -> 1,000
altman['재무구조'] = (df_share['시가총액']/1000) / df['[제조]부채(*)(IFRS연결)(천원)']

In [113]:
# 시가총액 계산시 0 인 값 확인할 것
print(f'시가총액 0인 row수 : {(df_share["시가총액"] == 0).sum()}') 
print(f'부채 0인 row수 : {(df["[제조]부채(*)(IFRS연결)(천원)"] == 0).sum()}')

# 시가총액 결측치 확인할 것
altman.isna().sum()


시가총액 0인 row수 : 2
부채 0인 row수 : 1


회사명               0
거래소코드             0
회계년도              0
총자산(log)       1868
총자산회전율(log)    1880
누적수익성          1880
재무구조           3998
K2_score       4005
dtype: int64

## III-II. K2 Score 적용
k2_score = -18.696 + 1.501 X1 + 2.706 X2 + 19.760 X3 + 1.146 X4

1. X1 : 기업규모 = log(총자산)
2. X2 : 총자산회전율 = log(매출액/총자산)
3. X3 : 누적수익성 = 이익잉여금 / 총자산
4. X4 : 재무구조 = 자기자본 시장가격 / 총부채
---
- i) K2 < -2.30 : 부실가능성 심각
- ii) -2.3 < K2 < 0.75 : 판정보류
- iii) K2 > 0.75 : 건전판정
---

In [114]:
altman.columns

Index(['회사명', '거래소코드', '회계년도', '총자산(log)', '총자산회전율(log)', '누적수익성', '재무구조',
       'K2_score'],
      dtype='object')

In [115]:
k2_series = -18.696 + 1.501*altman['총자산(log)'] + 2.706*altman['총자산회전율(log)'] + 19.760*altman['누적수익성'] + 1.146*altman['재무구조'] 

In [116]:
altman['K2_score'] = k2_series

In [117]:
altman

,회사명,거래소코드,회계년도,총자산(log),총자산회전율(log),누적수익성,재무구조,K2_score
0,(주)CMG제약,58820,2011/12,10.214832,-0.430596,-1.655914,8.521641,-27.483781
1,(주)CMG제약,58820,2012/12,10.952700,-1.256520,-0.917209,18.071527,-3.070217
2,(주)CMG제약,58820,2013/12,10.931544,-1.008149,-0.933631,85.360528,74.358806
3,(주)CMG제약,58820,2014/12,10.969376,-0.936942,-0.902280,88.553423,78.886829
4,(주)CMG제약,58820,2015/12,10.942577,-0.737434,-0.997891,69.860280,56.074872
...,...,...,...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,11.227981,0.793016,0.453301,17.886710,29.758492
17483,흥구석유(주),24060,2019/12,11.334534,0.588305,0.485883,4.504342,14.672119
17484,흥구석유(주),24060,2020/12,11.335249,0.316078,0.516579,9.845522,20.664077
17485,흥구석유(주),24060,2021/12,11.322335,0.469840,0.514773,13.696070,25.437823


In [118]:
# K2 score 범위 확인 

## 부실 가능성 심각
bad = (altman['K2_score'] < (-2.3)).sum()

## 판정 보류
neutral = ((altman['K2_score'] >= -2.3) & (altman['K2_score']<=0.75)).sum()

## 건전판정
good = (altman['K2_score'] > 0.75).sum()

###
agg = bad + neutral + good

print(f'부실 비중 : {(bad/agg)*100}')
print(f'보류 비중 : {(neutral/agg)*100}')
print(f'건전 비중 : {(good/agg)*100}')

부실 비중 : 18.001780151312865
보류 비중 : 7.669485239578698
건전 비중 : 74.32873460910844


In [119]:
bad

2427

In [120]:
neutral


1034

In [121]:
good

10021

In [122]:
altman['K2_score'].isna().sum()

4005

In [124]:
altman.to_excel('./dataset_2nd/altman_k2score.xlsx')